# 0️⃣ Instalar dependencias

In [ ]:
!pip install -q transformers[sentencepiece] datasets accelerate -U sentencepiece tokenizers sacremoses

import pandas as pd
from datasets import Dataset
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
import pickle
import shutil
import os
from google.colab import files
import torch

# 1️⃣ Subir dataset CSV

In [ ]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Usamos el separador | como en tu ejemplo
df = pd.read_csv(file_name, sep='|', quoting=3)
print(f"✅ Dataset cargado. Total filas: {len(df)}")
print(df.head(3))

# 2️⃣ Cargar MarianMT base y su Tokenizer

In [ ]:
# IMPORTANTE: Usamos el mismo tokenizer para TODO el proceso.
# El modelo 'itc-itc' ya está optimizado para lenguas romances (Latín incluído).
MODEL_NAME = "Helsinki-NLP/opus-mt-itc-itc"
tokenizer = MarianTokenizer.from_pretrained(MODEL_NAME)
model = MarianMTModel.from_pretrained(MODEL_NAME)
print(f"✅ Tokenizer cargado. Vocab size: {len(tokenizer.get_vocab())}")

# 3️⃣ Preparar Dataset BIDIRECCIONAL

In [ ]:
# Duplicamos los datos invirtiendo las columnas y añadiendo prefijos
df_la_es = df.copy()
df_la_es["src"] = ">>es<< " + df_la_es["latin"].astype(str)
df_la_es["tgt"] = df_la_es["spanish"].astype(str)

df_es_la = df.copy()
df_es_la["src"] = ">>la<< " + df_es_la["spanish"].astype(str)
df_es_la["tgt"] = df_es_la["latin"].astype(str)

df_final = pd.concat([df_la_es[["src", "tgt"]], df_es_la[["src", "tgt"]]], ignore_index=True)
print(f"✅ Dataset ampliado: {len(df_final)} filas (32k la->es + 32k es->la)")

# 4️⃣ Tokenizar dataset

In [ ]:
dataset = Dataset.from_pandas(df_final).shuffle(seed=42)
dataset_split = dataset.train_test_split(test_size=0.05)

def preprocess(examples):
    model_inputs = tokenizer(examples["src"], max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(text_target=examples["tgt"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_data = dataset_split.map(preprocess, batched=True)

# 5️⃣ Configurar entrenamiento

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./modelo_biblia_lat_es",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    num_train_epochs=8, # Ajusta según necesites
    learning_rate=3e-5,
    lr_scheduler_type="linear",
    warmup_steps=500,
    weight_decay=0.01,
    save_total_limit=3,
    predict_with_generate=True,
    fp16=True,
    logging_steps=200,
    evaluation_strategy="steps",
    eval_steps=500,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer
)

# 6️⃣ Entrenar modelo

In [ ]:
print("🚀 Iniciando entrenamiento...")
trainer.train()

# 7️⃣ Guardar TODO correctamente

In [ ]:
# save_pretrained guarda el modelo Y el tokenizer que realmente se usó.
model_dir = "./modelo_final_lat_es"
model.save_pretrained(model_dir, safe_serialization=False) # Guarda pytorch_model.bin + config.json
tokenizer.save_pretrained(model_dir) # Guarda vocab.json, source.spm, target.spm y configs

# Guardar training_args para referencia
with open(os.path.join(model_dir, "training_args.bin"), "wb") as f:
    pickle.dump(training_args, f)

# 8️⃣ Verificar y Comprimir

In [ ]:
essential_files = [
    "pytorch_model.bin",
    "config.json",
    "vocab.json",
    "source.spm",
    "target.spm",
    "tokenizer_config.json",
    "special_tokens_map.json"
]

print("\n🔍 Verificando archivos generados...")
for f in essential_files:
    path = os.path.join(model_dir, f)
    if os.path.exists(path):
        print(f"✅ {f} encontrado")
    else:
        print(f"❌ {f} NO encontrado")

zip_name = "marian_latin_spanish_model_CORRECTED"
shutil.make_archive(zip_name, 'zip', model_dir)
files.download(zip_name + ".zip")
print(f"\n🎉 ¡Listo! Descarga el archivo {zip_name}.zip")